# Create a mesh using `gmsh`
The aim here is to create a mesh where a material geometry in the form of a cube, surrounded by spherical air region, and finally a shell.

In [1]:
import gmsh
import math

Initiate mesh parameters.

In [2]:
cube_length = 100 # nm
inner_sphere_r = (cube_length * math.sqrt(3) / 2) + 2 # nm
outer_sphere_r = inner_sphere_r + 10 # nm
cube_length, inner_sphere_r, outer_sphere_r
l_mat = 5
l_rest = 10

In [3]:
gmsh.initialize()

In [4]:
gmsh.model.add("cube")

In [5]:
gmsh.logger.start()

Add a cube with origin at the centre of the cube.

In [6]:
cube = gmsh.model.occ.add_box(-cube_length/2, -cube_length/2, -cube_length/2, cube_length, cube_length, cube_length)

Add two spheres with the center at the origin.

In [7]:
inner_shpere = gmsh.model.occ.add_sphere(0, 0, 0, inner_sphere_r)

In [8]:
outer_shpere = gmsh.model.occ.add_sphere(0, 0, 0, outer_sphere_r)

Create a shell by removing the inner shere from of the outer sphere.

In [9]:
shell = gmsh.model.occ.cut([(3, outer_shpere)], [(3, inner_shpere)], removeTool=False)

In [10]:
shell[0]

[(3, 3)]

Create air region by removing cube from the inner sphere.

In [11]:
air = gmsh.model.occ.cut([(3, inner_shpere)], [(3, cube)], removeTool=False)

In [12]:
air[0]

[(3, 2)]

In [13]:
gmsh.model.occ.synchronize()

Create **physical groups** in order to select the regions easily later.

In [14]:
gmsh.model.add_physical_group(3, [shell[0][0][1]], name="shell")
gmsh.model.add_physical_group(3, [air[0][0][1]], name="air")
gmsh.model.add_physical_group(3, [cube], name="material")

3

Set the mesh size to a default for all points first.

In [15]:
gmsh.model.mesh.setSize(gmsh.model.get_entities(0), l_rest)

Reduce the size of the mesh for the points inside the material cude.

In [16]:
gmsh.model.mesh.setSize(
    gmsh.model.get_entities_in_bounding_box(
        -cube_length/2,
        -cube_length/2,
        -cube_length/2,
        cube_length/2,
        cube_length/2,
        cube_length/2
    ),
    l_mat
)

In [17]:
gmsh.model.mesh.generate(3)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 30%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 40%] Meshing curve 7 (Line)
Info    : [ 40%] Meshing curve 8 (Line)
Info    : [ 50%] Meshing curve 9 (Line)
Info    : [ 60%] Meshing curve 10 (Line)
Info    : [ 60%] Meshing curve 11 (Line)
Info    : [ 70%] Meshing curve 12 (Line)
Info    : [ 80%] Meshing curve 14 (Circle)
Info    : [ 90%] Meshing curve 17 (Circle)
Info    : Done meshing 1D (Wall 0.00306072s, CPU 0.005014s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 70%] M

Visualise the mesh.

In [18]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20240524
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [19]:
gmsh.write("cube.msh")

Info    : Writing 'cube.msh'...
Info    : Done writing 'cube.msh'


Inspect the log.

In [20]:
log = gmsh.logger.get()
print("Logger has recorded " + str(len(log)) + " lines")
gmsh.logger.stop()

Logger has recorded 196 lines


In [21]:
gmsh.finalize()